# Corrections and clarifications from yesterday
* Instructions for installing git
    * Make sure you install [git](https://git-scm.com/downloads), not the GitHub client
    * https://git-scm.com/downloads
    * Once you've done that, open a terminal window (command prompt on Windows)
    * type: `git clone https://github.com/tyarkoni/SSI2016.git`

* If you have trouble installing Rpy2, try:
    * `conda install -c r rpy2=2.7.0`
    * But note that we won't use it again in this course
* The Jupyter slideshow extension must be installed separately
    * Follow instructions here: https://github.com/damianavila/RISE

# Overview of Day 2
* Motivation
* A working dataset
* Importing data
* Pandas data structures
* Preprocessing data
* Working with text*

# Motivation
* Data preparation is the un-sexy part of data science
* But in many ways, the most important part
* Choice of statistical model often makes no practical difference
    * "More data beats better algorithms"
* Small differences in preprocessing choices can ramify quickly
    * E.g., variable smoothing or transformation, outlier removal, etc.

# Dataset
* The City of Austin has an [Open Data Portal](https://data.austintexas.gov/) containing many interesting datasets
* We'll use [outcome data](https://data.austintexas.gov/Health/Austin-Animal-Center-Outcomes/jpst-ix7f) from the Austin Animal Center
* 43,870 outcomes between October 2013, and February 2016
* Variables include the type, age, sex, breed, name, and color of the animal, plus the outcome

### Some questions we can ask
* What do people in Austin name their cats and dogs?
* What are the most common dog and cat breeds processed at the animal center?
* Do outcomes differ (e.g., euthanasia vs. adoption) for different breeds?
    * Are purebred dogs more likely to be adopted than mixes?
* Does animal size (e.g., bigger vs. smaller dogs) have any impact on outcomes?
* How well can we predict the likely outcome given everything we know about an animal?
* Do outcomes vary over time--e.g., by day of week, season, year, etc.?

## Import all the things

In [ ]:
# We'll consolidate all our imports at the top today
import re
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline

# You may need to install the packages below. You can:
# conda install beautifulsoup4 requests seaborn
from bs4 import BeautifulSoup
import requests
import seaborn as sns

# Disable annoying SettingWithCopyWarning
pd.options.mode.chained_assignment = None

# Importing data
* Before we do anything else, we need to get our data into a usable form
* Most commonly, data will come from a flat file
* But sometimes we need to retrieve data from other sources
* We'll do both

### Reading data in with the standard library
There are many ways to read in data in Python using the standard library. Here's a simple example, where we read in the data line-by-line and split each line into its own list.

In [ ]:
filename = '../data/Austin_Animal_Center_Outcomes.csv'
data = []  # Initialize an empty list to store the data

# Loop over rows in the file, split each one into a list
# of values, and add the result to the data list.
for line in open(filename).readlines():
    line = line.strip().split(',')
    data.append(line)

print("Found {} rows.".format(len(data)))

# Print the 1000th row to see what it looks like
data[1000]

The problem with approaches like the one above is that the data lack a tabular format, making it very hard to operate over rows or columns. We're much better off using the _pandas_ package to hold our data in a pandas DataFrame (DF)--a data structure that wraps around numpy arrays and is expressly designed to support a range of powerful operations over data. Reading a dataset into a pandas DF is very easy with the workhorse [read_csv()](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) or [read_table()](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_table.html) methods. These methods take a large number of optional arguments that make it easy to read in almost any kind of orderly data represented in a text file.

#### Reading data, the pandas way

In [ ]:
# Note that we're reading the file directly from GitHub.
# pandas accepts URLs in addition to local files.

url = 'https://raw.githubusercontent.com/tyarkoni/SSI2016/master/data/Austin_Animal_Center_Outcomes.csv'
# If you're working from the cloned course GitHub repo, comment the line above and uncomment
# the line below for faster loading.
# url = '../data/Austin_Animal_Center_Outcomes.csv'

# The workhorse data-reading method in pandas.
# It accepts a LOT of optional arguments--
# see http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html
data = pd.read_csv(url)

# calling head() on a DataFrame shows the top N rows.
data.head(5)   

## Other formats
Pandas has built-in support for [reading from or to other common formats/sources](http://pandas.pydata.org/pandas-docs/stable/io.html):
* Generic delimited text -- read_table()
* Excel -- read_excel()
* JSON -- read_json()
* SQL -- read_sql()
* Stata -- read_stata()
* SAS (XPORT or SAS7BDAT) -- read_sas()
* etc...

## Scraping data
* What if we want to add some data to our dataset?
* It would be nice if we had height and weight estimates for dog breeds
    * Are there different outcomes for bigger vs. smaller dogs?
* We track down a website that has some [breed information](http://www.wisdompanel.com/breed_count_matters/breedlisting/?browseby=size&F_All=Y)
* Now we need to "scrape" that data and get it into Python/pandas

In [ ]:
headers = { 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36'}
url = "http://www.wisdompanel.com/breed_count_matters/breedlisting/?browseby=size&F_All=Y"

html = requests.get(url, headers=headers).text
soup = BeautifulSoup(html, 'html.parser')

# Extract links to all breeds with BeautifulSoup
breed_links = soup.select('h3.itemTitle a')
breed_links = ["http://www.wisdompanel.com/" + a['href'] for a in breed_links]
print("Retrieving data for {} breeds.".format(len(breed_links)))

breed_data = []
for breed in breed_links:

    try:
        breed_html = requests.get(breed, headers=headers).text

        # Use regular expressions to extract name, height, width, description, and group
        name = re.search('<h3>AKC Name:</h3>(.*?)<', breed_html, re.DOTALL).group(1)
        min_weight, max_weight = re.search('<h3>Observed Weight:</h3>(\d+)[\s-]+(\d+)', breed_html, re.DOTALL).groups()
        height = re.search('Average Height:\s+(\d+)', breed_html, re.DOTALL).group(1)
        group = re.search('<h3>Genetic Group:</h3>(.*?)<', breed_html, re.DOTALL).group(1)
        breed_row = [name, min_weight, max_weight, height, group]
        print("Retrieved breed data: ", breed_row)
        breed_data.append(breed_row)
        
    # Some breed pages have formatting errors, so we'll just skip them
    except:
        pass

# Put the data in a pandas DataFrame
breed_data = pd.DataFrame(breed_data, columns=['breed_name', 'min_weight', 'max_weight', 'height', 'group'])

# Create output directory if it doesn't exist
output_dir = os.path.join('..', 'data')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the breed data locally for re-use
output_file = os.path.join(output_dir, 'breed_data.csv')
breed_data.to_csv(output_file, index=False, encoding='utf8')
print("Successfully saved breed data to {}.".format(output_file))

#### The scraped breed data
We'll come back later to the breed data we just scraped and saved. But for now, let's see what it looks like.

In [ ]:
breed_data = pd.read_csv('https://raw.githubusercontent.com/tyarkoni/SSI2016/master/data/breed_data.csv')

breed_data.head()

## Regular expressions
* The heavy lifting in the previous scraping code is done with regular expressions
* A powerful system for detecting and capturing patterns in text
* One of the most underutilized features of programming languages
* A short regular expression can replace dozens of lines of string-processing code
* Lots of good tutorials ([1](https://developers.google.com/edu/python/regular-expressions), [2](http://regexone.com/references/python), [3](http://www.learnpython.org/en/Regular_Expressions))

In [ ]:
# A snippet of HTML from the Alaskan Malamute's breed
# website on wisdompanel.com. We want to extract just the
# genetic group information.
text = '''
<div><h3>AKC Name:</h3>Alaskan Malamute<br /><br /><h3>Common Name(s):</h3>Malamute<br /><br />
<h3>AKC Group:</h3>Working<br /><br /><h3>Genetic Group:</h3>Asian<br /><br />
<h3>Observed Weight:</h3>61-105 lbs<br /><br /><h3>Show Weight:</h3>68-92 lbs</div>
'''

# Extract with regular expressions. In this case,
# We're looking to capture the group of all characters
# that occur after '<h3>Genetic Group:</h3>'
# and before '<'. We use "lazy" matching (the ? in
# the group .*?) to indicate that we want the regex
# engine to capture the _minimum_ amount of text possible
# rather then default greedy behavior, which captures
# as much text as possible.

pattern = '<h3>Genetic Group:</h3>(.*?)<'

# Search for pattern within text.
# The re.DOTALL flag indicates that we want to match
# across multiple lines of text, including newlines.
matches = re.search(pattern, text, re.DOTALL)

# Find the first match--i.e., the first group in our
# pattern enclosed in parentheses.
first_group = matches.group(1)

print(first_group)

In [ ]:
# Some very simple regex examples

text = "A few additional examples (maybe 4 or 5) illustrating the power of regular expressions!!"

# All text beginning with 'add' and ending in 'power'
print(re.search('(add.*power)',  text).group(1))

# All numbers in the text
print(re.findall('(\d+)', text))

# All words between 5 and 8 characters in length
print(re.findall(r'\b(\w{5,8})\b', text))

# Pandas data structures
* Provides functionality similar to data frames in R
* Two main data structures: Series and DataFrames
* A Series is a 1-dimensional numpy array with axis labels

In [ ]:
# Initialize a Series from a numpy array and index labels
a = np.arange(3, 8)
b = pd.Series(a, index=['apple', 'banana', 'orange', 'pear', 'grapes'])

# Let's take a look...
print(b)

In [ ]:
# Unlike numpy arrays, we can now refer to elements by label.
# The syntax is similar to dictionary indexing. You can also
# treat labels like attributes (e.g., b.pear), but this runs
# the risk of collisions and should be avoided.
print(b['pear'])

# We can always retrieve the underlying numpy array with .values
print(b.values)

# Many numpy operations work as expected, including slicing
print(b[2:4])

# Each column in our loaded dataset is a Series
print(data['Breed'][:5])

### The pandas DataFrame
* The workhorse of data analysis in pandas
* A container of multiple aligned Series
* Heterogeneous: a DF's Series can have different dtypes

#### Indexing pandas DataFrames
* pandas DFs spport [flexible indexing](http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing) by labels and/or indices
    * A common gotcha: R-style indexing won't work
    * Be explicit about whether you're using integer or label indexing

In [ ]:
# This won't work!
data[0, 'Animal Type']

# but .ix supports mixed integer and label based access
data.ix[0, 'Animal Type']

# Returns the entire column
data['Animal Type']

# Position-based selection; returns all of rows 2 - 5
data.iloc[2:5]

# Returns rows 2 - 5, columns 2 and 7
data.iloc[2:5, [2, 7]]

# Label-based indexing; equivalent to data['Animal Type']
# in this case
data.loc[:, 'Animal Type']

### A quick summary of the data
We'll cover descriptive analyses in more detail tomorrow, but for now, a basic summary:

In [ ]:
data.describe()

# Preprocessing our data
Much of what data scientists do involves cleaning and preprocessing data:
* Handling missing or invalid values
* Extracting usable information from messy strings
* Transforming/normalizing variables and variable names
* Filtering redundant or bad data
* Merging with other datasets
* Etc...

## Some examples
* There's far too much to cover in 2 hours, so we'll cherry-pick useful examples
* There are many excellent tutorials and guides (e.g., [1](http://www.gregreda.com/2013/10/26/intro-to-pandas-data-structures/), [2](http://synesthesiam.com/posts/an-introduction-to-pandas.html), [3](https://github.com/fonnesbeck/statistical-analysis-python-tutorial))

## Renaming/replacing
The column names in our dataset are kind of clunky. Let's make it a bit easier to work with the dataset by standardizing and shortening them.

In [ ]:
# This is no good
data.columns

In [ ]:
# Dictionary of old names --> new names.
# Notice that we don't need to provide replacements
# for _all_ the columns,  just the ones we want to
# replace. An alternative approach would be to
# directly set data.columns to a list of the
# desired new column names.
replacements = {
    'Animal ID': 'id',
    'DateTime': 'datetime',
    'Outcome Type': 'outcome',
    'Outcome Subtype': 'outcome_subtype',
    'Animal Type': 'animal',
    # We append these with '_string' because we're going
    # to use the names 'sex' and 'age' later for
    # extracted variables, so this avoids confusion.
    'Sex upon Outcome': 'sex_string',
    'Age upon Outcome': 'age_string',
}

# We can apply the rename function to either row names
# or column names. Here we're doing columns.
data = data.rename(columns=replacements)

# Let's also make sure all column names are in lowercase.
# We're using an idiom called a "list comprehension".
data.columns = [c.lower() for c in data.columns]

# The above list comprehension is functionally equivalent
# to the following for-loop:
# columns = []
# for c in data.columns:
#     columns.append(c)
# data.columns = columns

## Working with time series data
* The datetime column in our dataset is currently represented as a string
* We want to treat it as a datetime, so we can easily extract information
* Let's convert its time to datetime and pull out some useful columns

In [ ]:
# These are actually strings/objects, not datetimes!
print(data['datetime'][:3])

In [ ]:
# Convert the datetime column to actual datetimes
data['datetime'] = pd.to_datetime(data['datetime'])

# Now we're working with datetimes
print(data['datetime'][:3])

Now we can easily extract some useful time-related variables for later use (e.g., year, day of week, etc.).

In [ ]:
# Datetime-related functions are accessed through the .dt attribute
data['year'] = data['datetime'].dt.year
data['hour'] = data['datetime'].dt.hour
data['day'] = data['datetime'].dt.weekday

data.head(3)

## Parsing text data
* Many datasets contain nominal/categorical variables that need to be recoded as numeric variables
* Three common strategies:
    * Transform a nominal variable into a continuous one
    * Break down a compound nominal label into constituent numeric variables
    * "Dummy-code" a nominal variable as a set of binary variables

### Representing animal age with numeric values
The age column in our DF has nasty values like "3 years" and "7 months"

In [ ]:
data['age_string'].unique()

How do we convert these strings to numeric values we can operate on?

We can take advantage of pandas' apply() functionality, which applies arbitrary methods to each value in a Series (or each column or row in a DataFrame).

In [ ]:
def extract_months(value):
    ''' Extract and return age in months from string values 
    "3 years" or "7 months".
    
    Note: this code won't work as written, because some of
    the values passed in are invalid.
    
    Exercise: Modify the code to return the special value
    np.nan (numpy's representation of not-a-number)
    whenever an invalid value is encountered.
    '''
    number, unit = value.split(' ')
    number = int(number)

    if unit.startswith('year'):
        number *= 12

    return number

data['age'] = data['age_string'].apply(extract_months)

In [ ]:
# Notice any problems in the output? (Hint: week)
# As an exercise, you can fix this by tweaking the
# extract_months() method we wrote above and rerunning
# the previous code block. 
data[['age_string', 'age']][:10] 

### Just for kicks...
We're not ready to cover plotting just yet, but since we now have our first continuous variable, let's reward ourselves by plotting something nice to look at...

In [ ]:
sns.violinplot(x='animal', y='age', data=data, hue='outcome')
plt.gcf().set_size_inches((20, 6))

### Extracting data from the sex column
* The original column combines sex and sterilization status into one column

In [ ]:
data['sex_string'].value_counts()

Let's recode this as two binary variables: sex (male/female) and sterilized (True/False)

In [ ]:
# NOTE: The code below will fail the first time you run it. Why?
# Because there's a single NaN value that messes up the indexing.
# We need to recode the NaN--which we can do by setting it to
# 'Unknown', just like the other 3,625 Unknown values. To do
# that, uncomment the next line and re-run.

# data['sex_string'] = data['sex_string'].fillna('Unknown')

# Initialize a new column with NaN as the default value (for unknown sex)
data['sex'] = np.nan

# Update the column with values for M (0) and F (1)
data['sex'][data['sex_string'].str.contains('Male')] = 0
data['sex'][data['sex_string'].str.contains('Female')] = 1

# Now do the same kind of thing for sterilization
data['sterilized'] = 1
data['sterilized'][data['sex_string'].str.contains('Intact')] = 0
data['sterilized'][data['sex_string'].str.contains('Unknown')] = np.nan

In [ ]:
data[['sex_string', 'sex', 'sterilized']][:10]

Note: an alternative way to do the above would have been to use pandas' .replace() method. As an exercise, rewrite the code above to achieve the same goal using .replace(). (Hint: create a dict of values to find and replace.)

### Dummy-coded variables
* Categorical variables with N levels often need to be modeled as N binary indicators
* E.g., day-of-week is coded 0 through 6, but should be modeled as 6 or 7 contrasts
* pandas makes this easy with get_dummies()

In [ ]:
# We can also specify drop_first=True (on pandas >= 0.18) in order to
# use N - 1 columns instead of N (e.g., to reduce collinearity).
# It's also often useful to pass a prefix argument, which is prepended
# to all the level names.
pd.get_dummies(data['outcome'])[:5]

## Filtering data
* It's very common to want to filter out certain rows/columns

### Dropping uncommon outcomes
* Some outcomes in the dataset are too rare to merit analyzing
    * We already saw how extra levels clutter up our plots and understanding
* Let's impose a minimum cut-off of 200 occurrences for outcomes to be kept

In [ ]:
data['outcome'].value_counts()

#### A naive approach
How would we solve this problem in pure Python, without pandas?

In [ ]:
# Approach 1: the most naive approach, without
# taking advantage of anything pandas has to offer.
# This is slower and involves much more code than
# the pandas-based solutions, but it works just fine.
# To illustrate the logic behind what we're doing,
# we use only native Python here, and no fancy idioms.
# The other approaches do similar things under the
# hood, but take advantage of many optimizations and
# functions written in C to greatly speed things up.

# Get the raw outcome values as a list
values = data['outcome'].tolist()

# Initialize a dictionary to store running counts of
# all the outcomes.
counts = {}

# Loop over values and increment the appropriate
# outcome's counter.
for v in values:
    if v not in counts:
        counts[v] = 0
    counts[v] += 1

# Select the outcomes we want to keep--only those
# above 1000 occurrences.
min_count = 200

# Initialize a list to store only the valid outcomes
valid_outcomes = []

# Loop over the outcome-count pairs in the dictionary
# and keep only those above the threshold.
for outcome, count in counts.items():
    if count >= min_count:
        valid_outcomes.append(outcome)

# Identify the valid rows in the pandas DataFrame
rows_to_keep = []
for i, row in data.iterrows():
    if row['outcome'] in valid_outcomes:
        rows_to_keep.append(row)

# Turn this list of Series back into a DataFrame
filtered_data = pd.DataFrame(rows_to_keep)

# Count the number of rows we have left and print them out
n_rows = filtered_data.shape[0]
print("After filtering, we have {} rows in the dataset.".format(n_rows))

#### A better way
* The pure Python approach works, but it's slow and very verbose
* We can do much better with pandas

In [ ]:
# Approach 2: We do essentially the same thing, but now we
# take advantage of various built-in pandas methods.
# Notice how much shorter--and faster!--the solution is.

# Count the number of occurrences of each outcome
counts = data['outcome'].value_counts()

# Keep only those above a threshold--say 1000
valid_outcomes = counts[counts >= 200].index

# Now select only dataset rows with those outcomes
valid_rows = data['outcome'].isin(valid_outcomes)
filtered_data = data[valid_rows]

print("After filtering, we have {} rows in the dataset.".format(filtered_data.shape[0]))

#### An even better way
* That was much better. But we can do this even _more_ efficiently...
* We'll use the split-apply-combine pattern

#### The [split-apply-combine](https://www.jstatsoft.org/article/view/v040i01/v40i01.pdf) pattern
* A very common data processing strategy
    * Split the dataset into groups
    * Apply some operation(s) to each group
    * (Optionally) combine back into one dataset
* pandas provides powerful and fast tools for this

In [ ]:
# A simple example calculate the mean age of each animal species

# Split the dataset on animal. This creates a pandas GroupBy object.
groups = data.groupby('animal')

# pandas provides many built-in operations on groups--e.g., mean, count, etc.
# see http://pandas.pydata.org/pandas-docs/stable/groupby.html#groupby-object-attributes
groups['age'].mean()

# The above line can be more explicitly written as:
# groups['age'].aggregate('mean')

In [ ]:
# We can apply arbitrary methods to each group.
# Here we use a small "lambda function" (comparable to
# an anonymous function in JavaScript, R, or Matlab)
# to count the number of animals of each species
# that are older than 12 months.
groups['age'].apply(lambda x: (x > 12).sum())

# A functionally equivalent, but more explicit way of
# doing the above would be:

# def count_animals_older_than_12_months(x):
#     return (x > 12).sum()
#
# groups['age'].apply(count_animals_older_than_12_months)

#### The one-line solution
With the above pattern in mind, we can now turn back to our outcome-filtering problem and solve it in one line.

In [ ]:
# Approach 3: Use groupby() and a short lambda function.

filtered_data = data.groupby('outcome').filter(lambda x: len(x) >= 200)

print("After filtering, we have {} rows in the dataset.".format(filtered_data.shape[0]))

### Filtering animals for only the last outcome
* Some animals have multiple outcomes in the dataset

In [ ]:
# Count number of outcomes per animal
outcome_counts = data.groupby('id')['outcome'].count()
print(outcome_counts[:5])

# How many animals with more than one outcome?
outcome_counts[outcome_counts > 1].count()

* We probably want to analyze only the _last_ outcome for each animal
* This also becomes easy with the groupby strategy

In [ ]:
# Recall that we have a datetime column we can sort on,
# to put all outcomes in chronological order.
data = data.sort_values('datetime')

# Group on animal and then use the built-in last() function
filtered_data = data.groupby('id').last()

# Note the handy comma injection the Python format method provides
print("Before filtering: {:,} rows".format(len(data)))
print("After filtering: {:,} rows".format(len(filtered_data)))

## Merging datasets
* It's very common to need to combine data from different sources
* Remember the breed data we scraped?
* Let's merge it into our main DataFrame
    * Will allow us to test whether genetic group, weight, or height matter
* This is a more complicated operation

### The problem
* We want to merge or 'join' our outcome data with the breed data
* The obvious column to join on here is the breed name
* But the names don't line up! Compare:

In [ ]:
print(data['breed'].value_counts().index[:10])
print(breed_data['breed_name'].value_counts().index[:10])

Here's what we'll do:
* Drop terms like "mix", "shorthair" and "longhair" that don't occur in the official names
* For mixed breeds in our outcome data, we'll only use the first breed
* For the most common breeds, we'll manually recode names so that they align
* Note that all these steps are imperfect; our data will be noisy
    * When munging data, we often need to decide how good is good enough

In [ ]:
# Restrict analysis only to dogs, since that's all we have breed info for.
# Notice our use of the query function, which enables us to perform SQL-like
# queries over DataFrames.
dogs = data.query('animal == "Dog"')

# Let's prepare a 'merge_key' column in our outcome data that should line up
# as well as possible with the breed_name column in the breed_data DataFrame.
# We'll start by dropping all occurrences of 'Mix', 'Shorthair', etc.
dogs['merge_key'] = dogs['breed'].replace([' Mix', ' Shorthair', ' Longhair'], '', regex=True)

# Next, we'll keep only the first breed in cases where there are multiple given.
# For example, "Beagle/Labrador Retriever" would become just "Beagle".
dogs['merge_key'] = dogs['merge_key'].str.split('/').str.get(0)

# Manually map the most common breeds that don't align. I've cheated here
# and done a bit of extra work ahead of time to figure out what these are.
replacements = {
    "Pit Bull": "American Staffordshire Terrier",
    "Staffordshire": "Staffordshire Bull Terrier",
    "German Shepherd": "German Shepherd Dog",
    "Anatol Shepherd": "Anatolian Shepherd Dog",
    "Australian Shepherd": "Australian Shepherd Dog",
    "Catahoula": "Catahoula Leopard Dog",
    "American Pit Bull Terrier": "American Staffordshire Terrier", 
    "Doberman Pinsch": "Doberman Pinscher",  
}

# Apply the replacements
dogs['merge_key'] = dogs['merge_key'].replace(replacements)

# Finally, we're in a position to merge our data.
# We have to tell pandas which columns to use in each of the two datasets.
# We can also specify how we want the join to work ('left', 'outer',
# 'inner', etc.). The default is left, meaning that the output DataFrame
# will have the same number of rows as the first input DataFrame (dogs).
dogs = dogs.merge(breed_data, left_on='merge_key', right_on='breed_name', how='left')

In [ ]:
dogs.head(5)

# What else?

# Your (optional) assignment for tomorrow
* We extracted a numeric representation of several categorical variables
    * We could extract many more
* Some more variables we'll probably look at when we turn to fitting models:
    * Animal color
    * Whether the animal is a purebreed or mix
    * Whether the outcome resulted in an animal's death (e.g., Died + Euthanasia)
* Extract anything else you think might be useful